In [1]:
from astropy.time import Time, TimeDelta
import astropy.units as u
import glob
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sunpy.coordinates.sun as sn
import scipy.ndimage as ndi
import scipy.stats as st
# Our own library for using spice with STEREO (https://github.com/LukeBarnard/stereo_spice)
from stereo_spice.coordinates import StereoSpice
# Local packages
import HUXt as H

spice = StereoSpice()


In [15]:
def setup_huxt(start_time, wind='uniform'):
    """
    Initialise HUXt with some predetermined boundary/initial conditions
    start_time should be astropy.Time object.
    wind should be uniform or structured
    """
    cr_num = np.fix(sn.carrington_rotation_number(start_time))
    ert = H.Observer('EARTH', start_time)

    # Set up HUXt for a 5 day simulation with homogenous inner boundary.
    vr_in, br_in = H.Hin.get_MAS_long_profile(cr_num, ert.lat.to(u.deg))
    if wind == 'uniform':
        vr_in = np.zeros(vr_in.shape) + 400*vr_in.unit
        
    model = H.HUXt(v_boundary=vr_in, cr_num=cr_num, cr_lon_init=ert.lon_c, latitude=ert.lat.to(u.deg),
                   br_boundary=br_in, lon_start=270*u.deg, lon_stop=90*u.deg, simtime=3.5*u.day, dt_scale=4)
    
    return model

def get_base_cme(v=1000, lon=0, lat=0, width=30, thickness=5):
    """
    Return the base CME, which is used to establish the pseudo-truth CME and the SIR ensemble
    """
    t_launch = (1*u.hr).to(u.s)
    cme = H.ConeCME(t_launch=t_launch, longitude=lon*u.deg, latitude=lat*u.deg, width=width*u.deg, v=v*(u.km/u.s), thickness=thickness*u.solRad)
    return cme

def perturb_cone_cme(cme):
    """
    Perturb a ConeCME's parameters. Used to establish the pseudo-truth CME and the initial SIR ensemble members. 
    """
    lon_new = perturb_cme_param(cme.longitude, 15*u.deg)
    lat_new = perturb_cme_param(cme.latitude, 15*u.deg)
    width_new = perturb_cme_param(cme.width, 15*u.deg)
    speed_new = perturb_cme_param(cme.v, 150*u.km/u.s)
    thickness_new = perturb_cme_param(cme.thickness, 1*u.solRad)

    cme_perturb = H.ConeCME(t_launch=cme.t_launch,
                            longitude=lon_new,
                            latitude=lat_new,
                            width=width_new,
                            v=speed_new,
                            thickness=thickness_new)
    return cme_perturb

def perturb_cme_param(param, spread):
    """
    Randomly perturb a CME paramter with the uniform distribution and a specified spread
    """
    param_new = param + np.random.uniform(-1, 1, 1)[0] * spread
    return param_new

def twin_experiment():
    
    start_time = Time('2008-01-01T00:00:00')
    model = setup_huxt(start_time)
    
    cme_base = get_base_cme()
    
    cme_truth = perturb_cone_cme(cme_base)

    model.solve([cme_truth])
    
    cme_truth = model.cmes[0]
    obs = compute_observed_cme_profile(-60*u.deg, cme_truth)
    
    
def compute_observed_cme_profile(observer_lon, cme, dt_scale=5, el_max=30):
    """
    Compute the time-elongation profile of a specified observer.
    Observer longitude is relative to Earth.
    """
    
    # Compute the time-elongation profiles of the CME flanks from STA and STB
    coords = compute_t_e_profile(observer_lon, cme)

    # Remove invalid points
    coords.dropna(inplace=True)

    # Add observation noise.
    obs = coords.loc[:, ['time', 'el']].copy()
    obs['el'] = obs['el'] + 0.5*np.random.randn(obs.shape[0])

    # Only keep every dt_scale'th observation and reindex - dt_scale=5 corrsponds to ~2hr
    obs = obs[::dt_scale]
    obs.set_index(np.arange(0, obs.shape[0]), inplace=True)

    # Only analyse up to 30 deg elon ~ approx the HI1 FOV
    id_fov = obs['el'] <= el_max
    obs = obs[id_fov]
    return obs
    
def compute_t_e_profile(observer_lon, cme):
    """
    Compute the time elongation profile of the flank of a ConeCME in HUXt. The observer longtidue is specified relative to Earth,
    and but otherwise matches Earth's coords. 

    Parameters
    ----------
    observer_lon: Angular separation of Earth and the observer, in HEEQ.
    cme: A ConeCME object from a completed HUXt run (i.e the ConeCME.coords dictionary has been populated).
    Returns
    -------
    obs_profile: Pandas dataframe giving the coordinates of the ConeCME flank from STA's perspective, including the
                time, elongation, position angle, and HEEQ radius and longitude.
    """
    times = Time([coord['time'] for i, coord in cme.coords.items()])
    
    # Compute observers location using earth ephem, adding on observers longitude offset from Earth and correct for runover 2*pi
    obs = H.Observer('EARTH', times)
    obs.lon = obs.lon + observer_lon
    id_over = obs.lon > 2*np.pi*u.rad
    if np.any(id_over):
        obs.lon[id_over] = obs.lon[id_over] - 2*np.pi*u.rad
    
    obs_profile = pd.DataFrame(index=np.arange(times.size), columns=['time', 'el', 'r', 'lon'])
    obs_profile['time'] = times.jd
    
    for i, coord in cme.coords.items():

        if len(coord['r']) == 0:
            obs_profile.loc[i, ['lon','r', 'el']] = np.NaN
            continue

        r_obs = obs.r[i]
        x_obs = obs.r[i] * np.cos(obs.lat[i]) * np.cos(obs.lon[i])
        y_obs = obs.r[i] * np.cos(obs.lat[i]) * np.sin(obs.lon[i])
        z_obs = obs.r[i] * np.sin(obs.lat[i])

        lon_cme = coord['lon']
        lat_cme = coord['lat']
        r_cme = coord['r']

        x_cme = r_cme * np.cos(lat_cme) * np.cos(lon_cme)
        y_cme = r_cme * np.cos(lat_cme) * np.sin(lon_cme)
        z_cme = r_cme * np.sin(lat_cme)
        #############
        # Compute the observer CME distance, S, and elongation

        x_cme_s = x_cme - x_obs
        y_cme_s = y_cme - y_obs
        z_cme_s = z_cme - z_obs
        s = np.sqrt(x_cme_s**2 + y_cme_s**2 + z_cme_s**2)

        numer = (r_obs**2 + s**2 - r_cme**2).value
        denom = (2.0 * r_obs * s).value
        e_obs = np.arccos(numer / denom)

        # Find the flank coordinate and update output
        id_obs_flank = np.argmax(e_obs)       
        obs_profile.loc[i, 'lon'] = lon_cme[id_obs_flank].value
        obs_profile.loc[i, 'r'] = r_cme[id_obs_flank].value
        obs_profile.loc[i, 'el'] = np.rad2deg(e_obs[id_obs_flank])
    
    # Force values to be floats.
    keys = ['lon', 'r', 'el']
    obs_profile[keys] = obs_profile[keys].astype(np.float64)
    return obs_profile

Files already exist for CR2065
